In [1]:
# Importação das Bibliotecas e Parâmetros de Otimização
from functions import *
%matplotlib inline
sns.set(style="whitegrid", font_scale=1.2)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.size'] = '14'
plt.rcParams['figure.figsize'] = [10, 5]
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.2f' % x) # Tira os números do formato de Notação Científica
np.set_printoptions(suppress=True) # Tira os números do formato de Notação Científica em Numpy Arrays
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning) # Retira Future Warnings

# Spark Session
# from pyspark.sql import SparkSession, Row 
# #from pyspark.sql.functions import * as F
# import pyspark.sql.functions as F
# spark = SparkSession.builder.getOrCreate()

In [ ]:
'registros_publicos_depreciativos', 'consultas_credito_6meses', 'inadimplencia_vencida_30dias', 'consultas_credito_6meses'

In [2]:
def numero_de_anos_emprego_atual(df):
    df['qt_anos_mesmo_emprego'] = (df['qt_anos_mesmo_emprego'].replace({'< 1 year':0, '1 year':1, '2 years':2, '3 years':3, '4 years':4, '5 years':5, '6 years':6, '7 years':7, '8 years':8, '9 years':9,'10+ years':10}).fillna(0))
    df['qt_anos_mesmo_emprego'] = df['qt_anos_mesmo_emprego'].apply(lambda x:int(x))
    df['qt_anos_mesmo_emprego'] = np.where(df['qt_anos_mesmo_emprego'] <= 3, '3_YEARS', 
                        np.where(df['qt_anos_mesmo_emprego'] <= 6, '6_YEARS',
                        np.where(df['qt_anos_mesmo_emprego'] <= 9, '9_YEARS',
                        '10_YEARS+')))
    return df['qt_anos_mesmo_emprego']

def numero_de_registros_negativos(df):

    df = df[['situacao_do_emprestimo', 'registros_publicos_depreciativos']].copy()
    df[['registros_publicos_depreciativos']] = np.where(df[['registros_publicos_depreciativos']] == 0, 'sem_registros_negativos', 'com_registros_negativos')

    return df['registros_publicos_depreciativos']

def consulta_de_credito_nos_ultimos_6_meses(df):
    df = df[['situacao_do_emprestimo', 'consultas_credito_6meses']].copy()
    df[['consultas_credito_6meses']] = np.where(df[['consultas_credito_6meses']] == 0, 'sem_consultas', 'com_consultas')

    return df['consultas_credito_6meses']

def compromento_de_renda(df): 
    df_aux = df[['faturamento_anual', 'pagamento_mensal', 'valor_emprestimo_solicitado', 'qt_parcelas', 'taxa_de_juros', 'situacao_do_emprestimo']].copy()
    df_aux['qt_parcelas'] = np.where(df_aux['qt_parcelas'] == ' 36 months', 36, 60)
    df_aux['valor_emprestimo_solicitado_com_taxa_de_juros'] = df_aux['pagamento_mensal']*df_aux['qt_parcelas']
    df_aux['pagamento_anual'] = np.where(df_aux['qt_parcelas'] == ' 36 months', df_aux['valor_emprestimo_solicitado_com_taxa_de_juros']/3, df_aux['valor_emprestimo_solicitado_com_taxa_de_juros']/5)
    df_aux['comprometimento_de_renda_anual'] = ((df_aux['pagamento_anual']/df_aux['faturamento_anual'])*100).round(2)
    
    return df_aux['comprometimento_de_renda_anual']

def numero_incidencias_inadimplencia_vencidas_30d(df):
    df_aux = df[['situacao_do_emprestimo', 'inadimplencia_vencida_30dias']].copy()
    df_aux['inadimplencia_vencida_30dias'] = np.where(df_aux[['inadimplencia_vencida_30dias']] == 0, 'sem_inadimplencia_vencida', 'com_inadimplencia_vencida')

    return df_aux['inadimplencia_vencida_30dias']

def n_meses_produto_credito_atual(df):
    df = df.copy()
    df['data_financiamento_emprestimo'] = pd.to_datetime(df['data_financiamento_emprestimo'], format = '%b-%y')
    df['mths_since_data_financiamento_emprestimo'] = round(pd.to_numeric((pd.to_datetime('2023-09-20') - df['data_financiamento_emprestimo'])/np.timedelta64(1, 'M')))
    df['mths_since_data_financiamento_emprestimo'] = df['mths_since_data_financiamento_emprestimo'].fillna(df['mths_since_data_financiamento_emprestimo'].median())
    df['mths_since_data_financiamento_emprestimo'] = np.where(df['mths_since_data_financiamento_emprestimo'] < 0, df['mths_since_data_financiamento_emprestimo'].median(), df['mths_since_data_financiamento_emprestimo'])
    df['mths_since_data_financiamento_emprestimo'] = df['mths_since_data_financiamento_emprestimo'].apply(lambda x:int(x))
    df['data_financiamento_emprestimo'] = df['mths_since_data_financiamento_emprestimo']

    return df['data_financiamento_emprestimo']

def n_meses_primeiro_produto_credito(df):
    df = df.copy()
    df['data_contratacao_primeiro_produto_credito'] = pd.to_datetime(df['data_contratacao_primeiro_produto_credito'], format = '%b-%y')
    df['mths_since_data_contratacao_primeiro_produto_credito'] = round(pd.to_numeric((pd.to_datetime('2023-09-20') - df['data_contratacao_primeiro_produto_credito'])/np.timedelta64(1, 'M')))
    df['mths_since_data_contratacao_primeiro_produto_credito'] = df['mths_since_data_contratacao_primeiro_produto_credito'].fillna(df['mths_since_data_contratacao_primeiro_produto_credito'].median())
    df['mths_since_data_contratacao_primeiro_produto_credito'] = np.where(df['mths_since_data_contratacao_primeiro_produto_credito'] < 0, df['mths_since_data_contratacao_primeiro_produto_credito'].median(), df['mths_since_data_contratacao_primeiro_produto_credito'])
    df['mths_since_data_contratacao_primeiro_produto_credito'] = df['mths_since_data_contratacao_primeiro_produto_credito'].apply(lambda x:int(x))
    df['data_contratacao_primeiro_produto_credito'] = df['mths_since_data_contratacao_primeiro_produto_credito']
    
    return df['data_contratacao_primeiro_produto_credito']

def produto_disponivel_publicamente(df):
    df_aux = df[['situacao_do_emprestimo', 'produto_disponivel_publicamente']].copy()
    df_aux['produto_disponivel_publicamente'] = np.where(df_aux[['produto_disponivel_publicamente']] == 0, 'sem_disponibilidade_publica', 'com_disponibilidade_publica')

    return df_aux['produto_disponivel_publicamente']

def formato_features_binarias(df):
    df['qt_parcelas'] = np.where(df['qt_parcelas'] == ' 36 months', 0, 1)
    df['registros_publicos_depreciativos'] = np.where(df['qt_parcelas'] == 'sem_registros_negativos', 0, 1)
    df['inadimplencia_vencida_30dias'] = np.where(df['inadimplencia_vencida_30dias'] == 'sem_inadimplencia_vencida', 0, 1)
    df['tipo_de_concessao_do_credor'] = np.where(df['tipo_de_concessao_do_credor'] == 'f', 0, 1)
    df['plano_de_pagamento'] = np.where(df['plano_de_pagamento'] == 'n', 0, 1)
    df['renda_comprovada'] = np.where(df['renda_comprovada'] == 'Source Verified', 0, 1)
    df['consultas_credito_6meses'] = np.where(df['consultas_credito_6meses'] == 'sem_consultas', 0, 1)
    df['produto_disponivel_publicamente'] = np.where(df['consultas_credito_6meses'] == 'sem_disponibilidade_publica', 0, 1)

    return df

def target_encoder_bad_rate(df, tipo):
    categoricas = ['grau_de_emprestimo', 'subclasse_de_emprestimo', 'produto_de_credito', 'qt_anos_mesmo_emprego', 'status_propriedade_residencial', 'estado']
    df_aux_2 = df.copy()
    if tipo == 'Criação':
        for cat in categoricas:
            df_aux = df[[f'{cat}', 'situacao_do_emprestimo']].copy()
            good = pd.DataFrame(df_aux.loc[df_aux['situacao_do_emprestimo'] == 0].groupby(f'{cat}', as_index = False)['situacao_do_emprestimo'].count()).rename({'situacao_do_emprestimo':'qt_good'}, axis = 1)
            bad = pd.DataFrame(df_aux.loc[df_aux['situacao_do_emprestimo'] == 1].groupby(f'{cat}', as_index = False)['situacao_do_emprestimo'].count()).rename({'situacao_do_emprestimo':'qt_bad'}, axis = 1)
            df_aux = good.merge(bad, on = f'{cat}', how = 'left')
            df_aux['qt_total'] = df_aux['qt_good'] + df_aux['qt_bad']
            df_aux[f'{cat}_enc'] = ((df_aux['qt_bad']/df_aux['qt_total'])*100).round(2)
            df_aux[f'{cat}_enc'] = df_aux[f'{cat}_enc'].apply(lambda x:float(x))
            df_aux = df_aux[[f'{cat}', f'{cat}_enc']].drop_duplicates().sort_values(by = f'{cat}_enc', ascending = True)
            df_aux.to_csv(f'features/{cat}_enc.csv', index = False)
            df_aux_2 = df_aux_2.merge(df_aux[[f'{cat}', f'{cat}_enc']], on = f'{cat}', how = 'left')
            df_aux_2.drop(f'{cat}', axis = 1, inplace = True)
    else:
        for cat in categoricas:
            ft = pd.read_csv(f'features/{cat}_enc.csv')
            replace_dict = dict(zip(ft[f'{cat}'], ft[f'{cat}_enc']))
            df_aux_2[f'{cat}_enc'] = df_aux_2[f'{cat}'].replace(replace_dict)

    return df_aux_2

In [8]:
df = pd.read_parquet('data/credit_risk_loan_data.parquet').drop(['id', 'url'], axis = 1)
df = df[[
    "loan_status", "acc_now_delinq", "term", "grade", "sub_grade", "purpose", "loan_amnt", "int_rate", "issue_d", "policy_code", "pymnt_plan",
    "initial_list_status", "installment", "emp_title", "emp_length", "home_ownership", "verification_status", "delinq_2yrs", 
    "annual_inc", "addr_state", "tot_cur_bal", "total_rev_hi_lim", "revol_bal", "revol_util", "open_acc", "total_acc", "pub_rec",
    "inq_last_6mths", "earliest_cr_line", "mths_since_last_record", "mths_since_last_major_derog", "mths_since_last_delinq"
]]

df.rename(
    {
        "loan_status": "situacao_do_emprestimo",
        "acc_now_delinq": "qt_contas_inadimplentes_outras_instituicoes",
        "term" : "qt_parcelas",
        "grade" : "grau_de_emprestimo",
        "sub_grade" : "subclasse_de_emprestimo",
        "purpose" : "produto_de_credito",
        "loan_amnt" : "valor_emprestimo_solicitado",
        "int_rate" : "taxa_de_juros",
        "issue_d" : "data_financiamento_emprestimo",
        "policy_code" : "produto_disponivel_publicamente",
        "pymnt_plan" : "plano_de_pagamento",
        "initial_list_status" : "tipo_de_concessao_do_credor",
        "installment" : "pagamento_mensal",
        "emp_title" : "cargo_cliente",
        "emp_length" : "qt_anos_mesmo_emprego",
        "home_ownership" : "status_propriedade_residencial",
        "verification_status" : "renda_comprovada",
        "delinq_2yrs" : "inadimplencia_vencida_30dias",
        "annual_inc" : "faturamento_anual",
        "addr_state" : "estado",
        "tot_cur_bal" : "limite_total_produtos_credito",
        "total_rev_hi_lim" : "limite_total_rotativos",
        "revol_bal" : "limite_rotativos_utilizado",
        "revol_util" : "taxa_utilizacao_limite_rotativos",
        "open_acc" : "qt_produtos_credito_contratados_atualmente",
        "total_acc" : "qt_produtos_credito_contratados_historicamente",
        "pub_rec" : "registros_publicos_depreciativos",
        "inq_last_6mths" : "consultas_credito_6meses",
        "earliest_cr_line" : "data_contratacao_primeiro_produto_credito",
        "mths_since_last_record" : "qt_meses_desde_ultimo_registro_publico",
        "mths_since_last_major_derog" : "qt_meses_classificacao_mais_recente_90dias",
        "mths_since_last_delinq" : "qt_meses_ultima_inadimplencia"
    },
    axis = 1,
    inplace = True
)
df['situacao_do_emprestimo'] = np.where(df['situacao_do_emprestimo'].isin(['Charged Off', 'Default', 'Late (31-120 days)', 'Does not meet the credit policy. Status:Charged Off']) | df['qt_contas_inadimplentes_outras_instituicoes'].fillna(0) != 0, 'BAD', 'GOOD') 
df['situacao_do_emprestimo'] = np.where(df['situacao_do_emprestimo'] == 'BAD', 1, 0)

columns_selected = ['situacao_do_emprestimo', 'qt_parcelas','grau_de_emprestimo','subclasse_de_emprestimo','produto_de_credito', 'inadimplencia_vencida_30dias', 'valor_emprestimo_solicitado','taxa_de_juros','data_financiamento_emprestimo','produto_disponivel_publicamente','plano_de_pagamento','tipo_de_concessao_do_credor','pagamento_mensal','qt_anos_mesmo_emprego','status_propriedade_residencial',
'renda_comprovada','faturamento_anual','estado', 'limite_total_produtos_credito','limite_total_rotativos','limite_rotativos_utilizado','taxa_utilizacao_limite_rotativos','qt_produtos_credito_contratados_atualmente','qt_produtos_credito_contratados_historicamente','registros_publicos_depreciativos','consultas_credito_6meses','data_contratacao_primeiro_produto_credito','qt_meses_desde_ultimo_registro_publico', 'qt_meses_classificacao_mais_recente_90dias',
'qt_meses_ultima_inadimplencia']

df = df[columns_selected]

df_train_raw = separa_treino_teste('situacao_do_emprestimo', df, 0.20)[0]
df_test_raw = separa_treino_teste('situacao_do_emprestimo', df, 0.20)[1] 


df_train_silver = df_train_raw[columns_selected]
df_test_silver= df_test_raw[columns_selected]

df_test_silver['registros_publicos_depreciativos'] = numero_de_registros_negativos(df_test_silver)
df_test_silver['consultas_credito_6meses'] = consulta_de_credito_nos_ultimos_6_meses(df_test_silver)
df_test_silver['inadimplencia_vencida_30dias'] = numero_incidencias_inadimplencia_vencidas_30d(df_test_silver)

In [3]:
df = pd.read_parquet('data/credit_risk_loan_data.parquet').drop(['id', 'url'], axis = 1)
df = df[[
    "loan_status", "acc_now_delinq", "term", "grade", "sub_grade", "purpose", "loan_amnt", "int_rate", "issue_d", "policy_code", "pymnt_plan",
    "initial_list_status", "installment", "emp_title", "emp_length", "home_ownership", "verification_status", "delinq_2yrs", 
    "annual_inc", "addr_state", "tot_cur_bal", "total_rev_hi_lim", "revol_bal", "revol_util", "open_acc", "total_acc", "pub_rec",
    "inq_last_6mths", "earliest_cr_line", "mths_since_last_record", "mths_since_last_major_derog", "mths_since_last_delinq"
]]

df.rename(
    {
        "loan_status": "situacao_do_emprestimo",
        "acc_now_delinq": "qt_contas_inadimplentes_outras_instituicoes",
        "term" : "qt_parcelas",
        "grade" : "grau_de_emprestimo",
        "sub_grade" : "subclasse_de_emprestimo",
        "purpose" : "produto_de_credito",
        "loan_amnt" : "valor_emprestimo_solicitado",
        "int_rate" : "taxa_de_juros",
        "issue_d" : "data_financiamento_emprestimo",
        "policy_code" : "produto_disponivel_publicamente",
        "pymnt_plan" : "plano_de_pagamento",
        "initial_list_status" : "tipo_de_concessao_do_credor",
        "installment" : "pagamento_mensal",
        "emp_title" : "cargo_cliente",
        "emp_length" : "qt_anos_mesmo_emprego",
        "home_ownership" : "status_propriedade_residencial",
        "verification_status" : "renda_comprovada",
        "delinq_2yrs" : "inadimplencia_vencida_30dias",
        "annual_inc" : "faturamento_anual",
        "addr_state" : "estado",
        "tot_cur_bal" : "limite_total_produtos_credito",
        "total_rev_hi_lim" : "limite_total_rotativos",
        "revol_bal" : "limite_rotativos_utilizado",
        "revol_util" : "taxa_utilizacao_limite_rotativos",
        "open_acc" : "qt_produtos_credito_contratados_atualmente",
        "total_acc" : "qt_produtos_credito_contratados_historicamente",
        "pub_rec" : "registros_publicos_depreciativos",
        "inq_last_6mths" : "consultas_credito_6meses",
        "earliest_cr_line" : "data_contratacao_primeiro_produto_credito",
        "mths_since_last_record" : "qt_meses_desde_ultimo_registro_publico",
        "mths_since_last_major_derog" : "qt_meses_classificacao_mais_recente_90dias",
        "mths_since_last_delinq" : "qt_meses_ultima_inadimplencia"
    },
    axis = 1,
    inplace = True
)
df['situacao_do_emprestimo'] = np.where(df['situacao_do_emprestimo'].isin(['Charged Off', 'Default', 'Late (31-120 days)', 'Does not meet the credit policy. Status:Charged Off']) | df['qt_contas_inadimplentes_outras_instituicoes'].fillna(0) != 0, 'BAD', 'GOOD') 
df['situacao_do_emprestimo'] = np.where(df['situacao_do_emprestimo'] == 'BAD', 1, 0)

columns_selected = ['situacao_do_emprestimo', 'qt_parcelas','grau_de_emprestimo','subclasse_de_emprestimo','produto_de_credito', 'inadimplencia_vencida_30dias', 'valor_emprestimo_solicitado','taxa_de_juros','data_financiamento_emprestimo','produto_disponivel_publicamente','plano_de_pagamento','tipo_de_concessao_do_credor','pagamento_mensal','qt_anos_mesmo_emprego','status_propriedade_residencial',
'renda_comprovada','faturamento_anual','estado', 'limite_total_produtos_credito','limite_total_rotativos','limite_rotativos_utilizado','taxa_utilizacao_limite_rotativos','qt_produtos_credito_contratados_atualmente','qt_produtos_credito_contratados_historicamente','registros_publicos_depreciativos','consultas_credito_6meses','data_contratacao_primeiro_produto_credito','qt_meses_desde_ultimo_registro_publico', 'qt_meses_classificacao_mais_recente_90dias',
'qt_meses_ultima_inadimplencia']

df = df[columns_selected]

df_train_raw = separa_treino_teste('situacao_do_emprestimo', df, 0.20)[0]
df_test_raw = separa_treino_teste('situacao_do_emprestimo', df, 0.20)[1] 


df_train_silver = df_train_raw[columns_selected]
df_test_silver= df_test_raw[columns_selected]

df_test_silver['registros_publicos_depreciativos'] = numero_de_registros_negativos(df_test_silver)
df_test_silver['consultas_credito_6meses'] = consulta_de_credito_nos_ultimos_6_meses(df_test_silver)
df_test_silver['inadimplencia_vencida_30dias'] = numero_incidencias_inadimplencia_vencidas_30d(df_test_silver)

df_aux = df_test_silver.copy()

df_aux['qt_anos_mesmo_emprego'] = numero_de_anos_emprego_atual(df_aux)
df_aux['comprometimento_de_renda_anual'] = compromento_de_renda(df_aux)
df_aux['data_financiamento_emprestimo'] = n_meses_produto_credito_atual(df_aux)
df_aux['data_contratacao_primeiro_produto_credito'] = n_meses_primeiro_produto_credito(df_aux)
df_aux = formato_features_binarias(df_aux)
df_aux = target_encoder_bad_rate(df_aux, 'escoragem')

# Feature Selection

features_selected = pd.read_csv('features/features_selected.csv')
features_selected = features_selected.loc[features_selected['importance'] > 0] 
features_selected = list(features_selected['feature'].unique()) + ['situacao_do_emprestimo']
df_aux = df_aux[features_selected].copy()

x = df_aux.drop('situacao_do_emprestimo', axis = 1)
y = df_aux[['situacao_do_emprestimo']]

clf_final = carregar_modelo_pickle('models/clf_final.pkl')

y_pred = clf_final.predict(x)
y_proba= clf_final.predict_proba(x)
teste_threshold = pd.DataFrame({'y_predict':y_pred, 'Proba Good':y_proba[:, 0]})
teste_threshold['y_predict_threshold'] = np.where(teste_threshold['Proba Good'] <= 0.3, 1, 0)
y_pred = teste_threshold['y_predict_threshold'].values

df_aux['Conceder_Credito_Nao_Conceder_Credito'] = y_pred

Modelo carregado de models/clf_final.pkl


In [4]:
import gradio as gr 
import joblib as jb 

def predict(
    comprometimento_de_renda_anual,
    taxa_utilizacao_limite_rotativos,
    limite_rotativos_utilizado,
    limite_total_produtos_credito,
    faturamento_anual,
    pagamento_mensal,
    data_financiamento_emprestimo,
    taxa_de_juros,
    limite_total_rotativos,
    qt_produtos_credito_contratados_historicamente,
    valor_emprestimo_solicitado,
    qt_meses_ultima_inadimplencia,
    qt_produtos_credito_contratados_atualmente,
    qt_meses_classificacao_mais_recente_90dias,

    produto_de_credito_enc,
    qt_anos_mesmo_emprego_enc,
    grau_de_emprestimo_enc,
    subclasse_de_emprestimo_enc,
    status_propriedade_residencial_enc,
    renda_comprovada_enc,
    consultas_credito_6meses_enc,
    tipo_de_concessao_do_credor_enc,
    qt_parcelas_enc,
    inadimplencia_vencida_30dias_enc,
    registros_publicos_depreciativos_enc,
    plano_de_pagamento_enc,
    estado_enc):

    clf_final = carregar_modelo_pickle('models/clf_final.pkl')
    p = clf_final.predict_proba([[comprometimento_de_renda_anual,
        taxa_utilizacao_limite_rotativos,
        limite_rotativos_utilizado,
        limite_total_produtos_credito,
        faturamento_anual,
        pagamento_mensal,
        data_financiamento_emprestimo,
        taxa_de_juros,
        limite_total_rotativos,
        qt_produtos_credito_contratados_historicamente,
        estado,
        valor_emprestimo_solicitado,
        qt_meses_ultima_inadimplencia,
        qt_produtos_credito_contratados_atualmente,
        subclasse_de_emprestimo,
        qt_meses_classificacao_mais_recente_90dias,
        produto_de_credito,
        qt_anos_mesmo_emprego,
        grau_de_emprestimo_enc,
        qt_meses_desde_ultimo_registro_publico,
        status_propriedade_residencial,
        renda_comprovada,
        consultas_credito_6meses,
        tipo_de_concessao_do_credor,
        qt_parcelas,
        inadimplencia_vencida_30dias,
        registros_publicos_depreciativos,
        plano_de_pagamento]])
    
    if p[0] <= 3:
        resposta = 'Concessão de Crédito Aceita!'
    else:
        resposta = 'Concessão de Crédito Recusada!'

    return resposta

In [5]:
df_test_silver

,situacao_do_emprestimo,qt_parcelas,grau_de_emprestimo,subclasse_de_emprestimo,produto_de_credito,inadimplencia_vencida_30dias,valor_emprestimo_solicitado,taxa_de_juros,data_financiamento_emprestimo,produto_disponivel_publicamente,plano_de_pagamento,tipo_de_concessao_do_credor,pagamento_mensal,qt_anos_mesmo_emprego,status_propriedade_residencial,renda_comprovada,faturamento_anual,estado,limite_total_produtos_credito,limite_total_rotativos,limite_rotativos_utilizado,taxa_utilizacao_limite_rotativos,qt_produtos_credito_contratados_atualmente,qt_produtos_credito_contratados_historicamente,registros_publicos_depreciativos,qt_consultas_credito_6meses,data_contratacao_primeiro_produto_credito,qt_meses_desde_ultimo_registro_publico,qt_meses_classificacao_mais_recente_90dias,qt_meses_ultima_inadimplencia
362514,0,60 months,C,C5,home_improvement,sem_inadimplencia_vencida,32500,14.99,Jul-14,1,n,f,773.01,< 1 year,MORTGAGE,Verified,65000.00,CA,6563.00,25100.00,1208,4.30,7.00,20.00,sem_registros_negativos,sem_consultas,Apr-02,NaN,NaN,NaN
288564,0,60 months,E,E4,credit_card,com_inadimplencia_vencida,11000,20.99,Oct-14,1,n,w,297.53,10+ years,MORTGAGE,Verified,60000.00,IN,107972.00,24000.00,13129,54.70,18.00,43.00,sem_registros_negativos,sem_consultas,Aug-03,NaN,NaN,3.00
213591,0,36 months,C,C2,debt_consolidation,sem_inadimplencia_vencida,30000,14.65,Jun-12,1,n,f,1034.83,1 year,MORTGAGE,Verified,175000.00,AZ,NaN,NaN,1906,22.40,9.00,38.00,sem_registros_negativos,sem_consultas,Mar-02,NaN,NaN,41.00
263083,0,60 months,C,C4,debt_consolidation,sem_inadimplencia_vencida,14400,14.49,Nov-14,1,n,w,338.74,2 years,MORTGAGE,Source Verified,66000.00,MI,139126.00,17400.00,12944,74.40,12.00,31.00,sem_registros_negativos,sem_consultas,Aug-08,NaN,NaN,NaN
165001,0,36 months,A,A5,debt_consolidation,sem_inadimplencia_vencida,15000,8.90,Feb-13,1,n,f,476.30,10+ years,MORTGAGE,Verified,85000.00,PA,179532.00,8700.00,5902,67.80,7.00,20.00,sem_registros_negativos,sem_consultas,Oct-86,NaN,NaN,39.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,0,36 months,B,B3,credit_card,sem_inadimplencia_vencida,8000,11.71,Dec-11,1,n,f,264.61,9 years,MORTGAGE,Verified,35000.00,NV,NaN,NaN,6142,46.50,7.00,8.00,sem_registros_negativos,sem_consultas,Apr-06,NaN,NaN,NaN
296284,0,60 months,B,B2,debt_consolidation,sem_inadimplencia_vencida,24000,10.15,Oct-14,1,n,w,511.71,4 years,RENT,Verified,95000.00,IL,53265.00,23600.00,15217,64.50,10.00,22.00,sem_registros_negativos,sem_consultas,Jun-86,NaN,NaN,59.00
61777,1,36 months,A,A5,debt_consolidation,sem_inadimplencia_vencida,35000,8.90,Nov-13,1,n,f,1111.37,3 years,MORTGAGE,Source Verified,150000.00,LA,328082.00,57600.00,27828,48.30,9.00,32.00,sem_registros_negativos,sem_consultas,Dec-95,NaN,NaN,NaN
91763,0,36 months,A,A5,debt_consolidation,sem_inadimplencia_vencida,18000,8.90,Sep-13,1,n,f,571.56,9 years,MORTGAGE,Verified,55000.00,AZ,211938.00,21188.00,14631,69.10,7.00,22.00,sem_registros_negativos,sem_consultas,Oct-92,NaN,NaN,NaN


In [6]:
categoricas = df_test_silver[[
    'produto_de_credito',
    'qt_anos_mesmo_emprego',
    'grau_de_emprestimo',
    'subclasse_de_emprestimo',
    'status_propriedade_residencial',
    'renda_comprovada',
    'consultas_credito_6meses',
    'tipo_de_concessao_do_credor',
    'qt_parcelas',
    'inadimplencia_vencida_30dias',
    'registros_publicos_depreciativos',
    'plano_de_pagamento',
    'estado']]
for cat in categoricas:
    print(f'{cat} Unique: {list(df_test_silver[cat].unique())}')
    categoria_avaliada = pd.read_csv(f'features/{cat}_enc.csv')
    dicionario_resultante = dict(zip(list(categoria_avaliada[f'{cat}'].values), list(categoria_avaliada[f'{cat}_enc'].values)))
    print(dicionario_resultante)
    print()
    print()

produto_de_credito Unique: ['home_improvement', 'credit_card', 'debt_consolidation', 'vacation', 'house', 'medical', 'small_business', 'car', 'other', 'major_purchase', 'wedding', 'moving', 'educational', 'renewable_energy']
{'credit_card': 8.98, 'car': 9.23, 'major_purchase': 9.74, 'home_improvement': 10.31, 'debt_consolidation': 11.65, 'vacation': 11.85, 'wedding': 11.87, 'medical': 13.63, 'renewable_energy': 13.88, 'other': 14.11, 'house': 14.54, 'moving': 16.35, 'educational': 19.82, 'small_business': 21.66}


qt_anos_mesmo_emprego Unique: ['< 1 year', '10+ years', '1 year', '2 years', '9 years', None, '3 years', '6 years', '4 years', '7 years', '8 years', '5 years']
{'10_YEARS+': 10.39, '9_YEARS': 11.54, '6_YEARS': 11.6, '3_YEARS': 11.76}


grau_de_emprestimo Unique: ['C', 'E', 'A', 'D', 'B', 'F', 'G']
{'A': 3.96, 'B': 8.17, 'C': 11.79, 'D': 15.82, 'E': 20.03, 'F': 25.05, 'G': 28.07}


subclasse_de_emprestimo Unique: ['C5', 'E4', 'C2', 'C4', 'A5', 'D4', 'B3', 'C1', 'E3', 'C3', 'D2

In [7]:
categoricas = df_test_silver[[
    'produto_de_credito',
    'qt_anos_mesmo_emprego',
    'grau_de_emprestimo',
    'subclasse_de_emprestimo',
    'status_propriedade_residencial',
    'renda_comprovada',
    'consultas_credito_6meses',
    'tipo_de_concessao_do_credor',
    'qt_parcelas',
    'inadimplencia_vencida_30dias',
    'registros_publicos_depreciativos',
    'plano_de_pagamento',
    'estado']]

In [ ]:
categoria_avaliada = pd.read_csv(f'features/produto_de_credito_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'produto_de_credito'].values), list(categoria_avaliada[f'produto_de_credito_enc'].values)))
keys_produto_credito = list(dicionario_resultante.keys())
values_produto_credito = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/qt_anos_mesmo_emprego_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'qt_anos_mesmo_emprego'].values), list(categoria_avaliada[f'qt_anos_mesmo_emprego_enc'].values)))
keys_produto_qt_anos_mesmo_emprego = list(dicionario_resultante.keys())
values_produto_qt_anos_mesmo_emprego = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/grau_de_emprestimo_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'grau_de_emprestimo'].values), list(categoria_avaliada[f'grau_de_emprestimo_enc'].values)))
keys_produto_grau_de_emprestimo = list(dicionario_resultante.keys())
values_produto_grau_de_emprestimo = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/subclasse_de_emprestimo_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'subclasse_de_emprestimo'].values), list(categoria_avaliada[f'subclasse_de_emprestimo_enc'].values)))
keys_produto_subclasse_de_emprestimo = list(dicionario_resultante.keys())
values_produto_subclasse_de_emprestimo = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/status_propriedade_residencial_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'status_propriedade_residencial'].values), list(categoria_avaliada[f'status_propriedade_residencial_enc'].values)))
keys_produto_status_propriedade_residencial = list(dicionario_resultante.keys())
values_produto_status_propriedade_residencial = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/renda_comprovada_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'renda_comprovada'].values), list(categoria_avaliada[f'renda_comprovada_enc'].values)))
keys_produto_renda_comprovada = list(dicionario_resultante.keys())
values_produto_renda_comprovada = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/consultas_credito_6meses_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'consultas_credito_6meses'].values), list(categoria_avaliada[f'consultas_credito_6meses_enc'].values)))
keys_produto_consultas_credito_6meses = list(dicionario_resultante.keys())
values_produto_consultas_credito_6meses= list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/tipo_de_concessao_do_credor_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'produto_de_credito'].values), list(categoria_avaliada[f'tipo_de_concessao_do_credor_enc'].values)))
keys_produto_tipo_de_concessao_do_credor = list(dicionario_resultante.keys())
values_produto_tipo_de_concessao_do_credor = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/qt_parcelas_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'qt_parcelas'].values), list(categoria_avaliada[f'qt_parcelas_enc'].values)))
keys_produto_qt_parcelas= list(dicionario_resultante.keys())
values_produto_qt_parcelas = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/produto_de_credito_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'produto_de_credito'].values), list(categoria_avaliada[f'produto_de_credito_enc'].values)))
keys_produto_credito = list(dicionario_resultante.keys())
values_produto_credito = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/produto_de_credito_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'produto_de_credito'].values), list(categoria_avaliada[f'produto_de_credito_enc'].values)))
keys_produto_credito = list(dicionario_resultante.keys())
values_produto_credito = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/produto_de_credito_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'produto_de_credito'].values), list(categoria_avaliada[f'produto_de_credito_enc'].values)))
keys_produto_credito = list(dicionario_resultante.keys())
values_produto_credito = list(dicionario_resultante.values())

categoria_avaliada = pd.read_csv(f'features/produto_de_credito_enc.csv')
dicionario_resultante = dict(zip(list(categoria_avaliada[f'produto_de_credito'].values), list(categoria_avaliada[f'produto_de_credito_enc'].values)))
keys_produto_credito = list(dicionario_resultante.keys())
values_produto_credito = list(dicionario_resultante.values())


demo = gr.Interface(
    fn = predict
    inputs = [
        'number',
        'number',
        'number', 
        'number',
        'number',
        'number',
        'number',
        'number',
        'number',
        'number',
        'number',
        'number',
        'number',
        'number',

        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),
        gr.Dropdown(choices = [0, 1]),      
    ],
    outputs = 'label'
)

In [36]:
produto_de_credito_enc,
qt_anos_mesmo_emprego_enc,
grau_de_emprestimo_enc,
subclasse_de_emprestimo_enc,
qt_meses_desde_ultimo_registro_publico,
status_propriedade_residencial_enc,
renda_comprovada_enc,
consultas_credito_6meses_enc,
tipo_de_concessao_do_credor_enc,
qt_parcelas_enc,
inadimplencia_vencida_30dias_enc,
registros_publicos_depreciativos_enc,
plano_de_pagamento_enc,
estado_enc

,0
comprometimento_de_renda_anual,float64
taxa_utilizacao_limite_rotativos,float64
limite_rotativos_utilizado,int64
limite_total_produtos_credito,float64
faturamento_anual,float64
pagamento_mensal,float64
data_financiamento_emprestimo,int64
taxa_de_juros,float64
limite_total_rotativos,float64
qt_produtos_credito_contratados_historicamente,float64


In [39]:
clf_final = carregar_modelo_pickle('models/clf_final.pkl')

Modelo carregado de models/clf_final.pkl
